In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
import os
import sqlite3
import psycopg2
from psycopg2.extras import execute_batch


sqlite_conn = sqlite3.connect("music_store.db")
sqlite_cursor = sqlite_conn.cursor()

pg_conn = psycopg2.connect(
    dbname=os.environ["DB_NAME"],
    user=os.environ["DB_USER"],
    password=os.environ["DB_PASSWORD"],
    host=os.environ["DB_HOST"],
    port=os.environ["DB_PORT"]
)
pg_cursor = pg_conn.cursor()

sqlite_cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [table[0] for table in sqlite_cursor.fetchall()]

for table_name in tables:
    print(f"🔄 Migrating table: {table_name}")

    sqlite_cursor.execute(f"PRAGMA table_info({table_name});")
    columns = sqlite_cursor.fetchall()
    column_names = [column[1] for column in columns]

    create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ("
    create_table_query += ", ".join(
        f"{column[1]} {column[2].replace('NVARCHAR', 'VARCHAR').replace('DATETIME', 'TIMESTAMP')}"
        for column in columns
    )
    create_table_query += ");"
    
    pg_cursor.execute(create_table_query)

    sqlite_cursor.execute(f"SELECT * FROM {table_name};")
    rows = sqlite_cursor.fetchall()

    if rows:
        placeholders = ", ".join(["%s"] * len(column_names))
        insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({placeholders})"
        
        execute_batch(pg_cursor, insert_query, rows, page_size=500) 

    print(f"✅ Migration complete for table: {table_name}")

pg_conn.commit()
pg_cursor.close()
pg_conn.close()
sqlite_cursor.close()
sqlite_conn.close()

print(" Migration Completed Successfully!")


🔄 Migrating table: Album
✅ Migration complete for table: Album
🔄 Migrating table: Artist
✅ Migration complete for table: Artist
🔄 Migrating table: Customer
✅ Migration complete for table: Customer
🔄 Migrating table: Employee
✅ Migration complete for table: Employee
🔄 Migrating table: Genre
✅ Migration complete for table: Genre
🔄 Migrating table: Invoice
✅ Migration complete for table: Invoice
🔄 Migrating table: InvoiceLine
✅ Migration complete for table: InvoiceLine
🔄 Migrating table: MediaType
✅ Migration complete for table: MediaType
🔄 Migrating table: Playlist
✅ Migration complete for table: Playlist
🔄 Migrating table: PlaylistTrack
✅ Migration complete for table: PlaylistTrack
🔄 Migrating table: Track
✅ Migration complete for table: Track
🎉 Migration Completed Successfully!
